<a href="https://colab.research.google.com/github/Junhojuno/keras-tutorial/blob/master/06_RNN_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Recurrent Neural Network의 기본구조
- sequence 구조의 데이터가 들어간다. (이점은 앞선 feedforward network와 동일 ; feedforward는 앞선 FCN, CNN 등을 의미)
- 차이가 있다면, sequence구조의 데이터를 한번에 처리하지 않는다.
- 네트워크(RNN)가 sequence의 원소를 차례대로 방문하는데, 이때 사용되는 개념이 loop와 state
- (time steps, features)의 2D tensor를 데이터셋으로 받는데,
- 1개의 데이터를 받으면 RNN을 통과시켜 출력을 내는데, 이 출력을 다음 time step의 state로 설정
- 1 timestep != 1 데이터 ; 1개의 데이터는 많은 timestep으로 이루어져 있다. 데이터 하나가 2D tensor형태

```python
###############################################################
state_t = 0 # intial state
for input_t in input_sequence:
    output_t = f(input_t, state_t)
    state_t = output_t # 출력이 다음 반복을 위한 state가 된다.
    
###############################################################
# 좀 더 자세히 써보자
state_t = 0
for input_t in input_sequence:
    output_t = activation(dot(W,input_t) + dot(U,state_t) + b) # 이부분이 좀 새롭다!(input과 state를 연결하여 output을 계산)
    state_t = output_t
###############################################################
```

In [2]:
# simpleRNN
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000 # 사용할 단어수
maxlen = 500
batch_size = 32

(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(input_train.shape, input_test.shape) # 그냥 데이터 자체는 list형태
print("훈련 시퀀스 : {}, 테스트 시퀀스 : {}".format(len(input_train), len(input_test)))

print("시퀀스 패딩 ; (samples x time)")
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print(input_train.shape, input_test.shape) # 단어 인덱스로 만들어줍니다. 리뷰를 [32 7 92 42 12 ....]형태로 만들어준다. ; 텍스트를 숫자 시퀀스 형태로!

17465344/17464789 [==============================] - 0s 0us/step
(25000,) (25000,)
훈련 시퀀스 : 25000, 테스트 시퀀스 : 25000
시퀀스 패딩 ; (samples x time)
(25000, 500) (25000, 500)
